In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import quote_plus
import requests
import time
import string

from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
from IPython.display import display
pd.set_option('display.max_colwidth', None)
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

In [109]:
name="Bharat Forge"
encoded = quote_plus(name)
url='http://www.google.com/search?q='+encoded+'&tbm=nws'
print(url)
#sentiment_analysis(url)

http://www.google.com/search?q=Bharat+Forge&tbm=nws


def sentiment_analysis(name,url):
    
    print(url)
    data=collect_ray_data(url)
    print('raw data:')
    display(data.head())
    
    selected_data=selecting_data(data)
    print('selected data:')
    display(selected_data.head())    #--------------------------
    
    links=selected_data['Link']
    status=[]
    
    # traversing each link
    for li in links:
        
        request =  requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        if request.status_code == 200:
            status.append("working")
        else:
            status.append("notworking")
    selected_data.loc[data['Time'] !="", 'Status'] = status

    print('after selection:')  #----------------------------------
    display(selected_data.head())
    
    required_data = selected_data[ selected_data['Status']=='working' ] 
    print('required data:')
    display(required_data.head())
    
    inner_data = inner_data_(required_data)
    print('inner data:')
    display(inner_data.head())
    
    clear_data = topic_related_text_extraction(inner_data)
    print('clear data:')
    display(clear_data.head())
    
    analytics_data=final_analysis(clear_data)
    print('analytics data:')
    store=analytics_data[['Text','Time','Link','Score','Result']]
    store.to_csv(name+".csv", index=False)

    display(analytics_data.head())

    overall_compound = analytics_data["Score"].mean()
    if overall_compound > 0.5:
        final_result = "Strongly Positive"
    elif overall_compound > 0.05:
        final_result = "Positive"
    elif overall_compound >= -0.05:
        final_result = "Neutral"
    else:
        final_result = "Negative"
    print(overall_compound,'------',final_result)
    return overall_compound,final_result

In [110]:
def collect_ray_data(url):
    #for traversing all pages in news.
    driver = webdriver.Chrome()
    driver.get(url)
    

    final_text=[]
    final_sub_text=[]
    final_time=[]
    final_link=[]
    final_in=[]
    i=2
    while True:
        print(i)
        i+=1            


        divs = driver.find_elements(By.XPATH,'//*[@id="rso"]/div/div/div')
        for div_ in divs:
            try:
                text = div_.find_element(By.XPATH, './/div/div/a/div/div[2]/div[2]').text
                sub_text = div_.find_element(By.XPATH, './/div/div/a/div/div[2]/div[3]').text
                time_ = div_.find_element(By.XPATH, './/div/div/a/div/div[2]/div[4]/div/span').text
                print('time',time_)
                time,result=check_time(time_)
                print('in')
                link = div_.find_elements(By.XPATH,'.//div/div/a')[0].get_attribute("href")
                final_text.append(text)
                final_sub_text.append(sub_text)
                final_time.append(time)
                final_link.append(link)

            except:
                print('no link')


          
        #to next page slide
        
        path='//*[@id="pnnext"]'
        next_=driver.find_elements(By.XPATH,path)
        
        try:
            next_url=next_[0].get_attribute("href")
            driver.get(next_url)
        except:
            break
        #time.sleep(5)

    print(len(final_link))
    print(len(final_text))
    print(len(final_sub_text))
    print(len(final_time))
    print(len(final_in))
       #combine text for whol
    
    #return final_text,final_sub_text,final_time,final_link
    
    driver.quit()
    data=pd.DataFrame({'Text':final_text[:],'Sub text':final_sub_text[:],'Time':final_time[:],'Link':final_link[:]})
    return data

from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd

def check_time(time_):
    print('input:', time_)

    try:
        # case: "26 Aug 2025"
        time = datetime.strptime(time_, "%d %b %Y").date()

    except:
        date_split = time_.split(' ')

        if len(date_split) < 2:
            time = pd.NaT

        else:
            value = date_split[0]
            unit = date_split[1]

            if unit == 'hours':
                time = date.today()

            elif unit == 'day':
                time = date.today() - timedelta(days=1)

            elif unit == 'days':
                time = date.today() - timedelta(days=int(value))

            elif unit == 'weeks':
                time = date.today() - timedelta(weeks=int(value))

            elif unit == 'month':
                time = date.today() - relativedelta(months=int(value))

            else:
                if unit == 'sept':
                    time = date(int(data_split[2]),9,int(value))

    if pd.isna(time):
        result = False
    else:
        check = lambda t: t >= date.today() - relativedelta(months=12)
        result = check(time)

    print("parsed time:", time, "result:", result)
    return time, result

data=collect_ray_data(url)
data

2
time 3 days ago
input: 3 days ago
parsed time: 2026-01-10 result: True
in
time 19 hours ago
input: 19 hours ago
parsed time: 2026-01-13 result: True
in
time 1 day ago
input: 1 day ago
parsed time: 2026-01-12 result: True
in
time 2 days ago
input: 2 days ago
parsed time: 2026-01-11 result: True
in
time 4 days ago
input: 4 days ago
parsed time: 2026-01-09 result: True
in
time 2 weeks ago
input: 2 weeks ago
parsed time: 2025-12-30 result: True
in
time 2 days ago
input: 2 days ago
parsed time: 2026-01-11 result: True
in
time 4 days ago
input: 4 days ago
parsed time: 2026-01-09 result: True
in
time 4 days ago
input: 4 days ago
parsed time: 2026-01-09 result: True
in
time 4 days ago
input: 4 days ago
parsed time: 2026-01-09 result: True
in
3
time 4 days ago
input: 4 days ago
parsed time: 2026-01-09 result: True
in
time 4 days ago
input: 4 days ago
parsed time: 2026-01-09 result: True
in
time 3 days ago
input: 3 days ago
parsed time: 2026-01-10 result: True
in
time 4 days ago
input: 4 days 

,Text,Sub text,Time,Link
0,bharat forge and agile robots sign mou to deploy ai led industrial automation and dark factories,"News: In a press release, Bharat Forge announced it has signed an MoU with Germany-based Agile Robots to jointly advance AI-led industrial...",2026-01-10,https://www.icicidirect.com/research/equity/trending-news/bharat-forge-and-agile-robots-sign-mou-to-deploy-ai-led-industrial-automation-and-dark-factories
1,Bharat Forge partners with Agile to advance AI-led automation,"Driven by Bharat Forge's domain expertise and Agile Robots' intelligent robots and automation solutions, this collaboration will see the...",2026-01-13,https://evertiq.com/design/2026-01-12-bharat-forge-partners-with-agile-to-advance-ai-led-automation
2,Bharat Forge Ltd. is Rated Buy by MarketsMOJO,"Bharat Forge Ltd. is rated Buy by MarketsMOJO, with this rating last updated on 15 Dec 2025. While the rating was revised on that date,...",2026-01-12,https://www.marketsmojo.com/news/stock-recommendation/bharat-forge-ltd-is-rated-buy-by-marketsmojo-3784356
3,Bharat Forge signs strategic MoU with Agile Robots,"Under the MoU, Bharat Forge and Agile Robots will jointly develop customised automation solutions for manufacturing, consumer goods and...",2026-01-11,https://www.manufacturingtodayindia.com/bharat-forge-with-agile-robots
4,"Stocks in news: RIL, BHEL, NTPC, Bharat Forge, Ashok Leyland","Markets closed lower on Thursday due to weak global cues, with the Nifty breaching a key support level. Several companies are in focus...",2026-01-09,https://m.economictimes.com/markets/stocks/news/stocks-in-news-ril-bhel-ntpc-bharat-forge-ashok-leyland/articleshow/126425250.cms
...,...,...,...,...
236,Bharat Forge to Tata Motors: Why did auto stocks fall in trade on Thursday?,"Shares of Bharat Forge and Samvardhana Motherson International fell over 3 per cent each, while Tata Motors and Balkrishna Industries fell over...",2025-07-31,https://www.business-standard.com/markets/news/bharat-forge-to-tata-motors-auto-stocks-fall-up-to-3-per-cent-trump-tariff-threat-125073100274_1.html
237,Bharat Forge acquires AAM India Manufacturing in a Rs 746 crore deal,Bharat Forge has acquired AAM India Manufacturing for Rs 746.4 crore. The acquisition includes the takeover of Rs 189.4 crore cash on the books of AAM India...,2025-07-02,https://www.financialexpress.com/business/industry-bharat-forge-acquires-aam-india-manufacturing-in-a-rs-746-crore-deal-3900474/
238,Nagaland Govt delegation visits Bharat Forge Ltd in Pune,"A delegation from the Nagaland government, including representatives from the Department of Industries & Commerce and the Nagaland Tool Room...",2025-08-23,https://nagalandtribune.in/nagaland-govt-delegation-visits-bharat-forge-ltd-in-pune/
239,Bharat Forge Limited completes Rs 746.46 crore acquisition of AAM India to boost auto and EV components,Bharat Forge Limited (BFL) has successfully completed the acquisition of AAM India Manufacturing Private Limited for an equity value of Rs 746.46 crore.,2025-07-03,https://www.zeebiz.com/companies/news-bharat-forge-limited-completes-rs-74646-crore-acquisition-of-aam-india-to-boost-auto-and-ev-components-371951


In [111]:
data['Time'].unique()

array([datetime.date(2026, 1, 10), datetime.date(2026, 1, 13),
       datetime.date(2026, 1, 12), datetime.date(2026, 1, 11),
       datetime.date(2026, 1, 9), datetime.date(2025, 12, 30),
       datetime.date(2026, 1, 7), datetime.date(2025, 12, 23),
       datetime.date(2025, 12, 13), datetime.date(2025, 10, 23),
       datetime.date(2025, 12, 16), datetime.date(2025, 10, 16),
       datetime.date(2025, 11, 13), datetime.date(2025, 11, 12),
       datetime.date(2025, 11, 11), datetime.date(2025, 10, 22),
       datetime.date(2025, 11, 14), datetime.date(2025, 8, 7),
       datetime.date(2025, 11, 17), datetime.date(2025, 10, 28),
       datetime.date(2025, 10, 13), datetime.date(2025, 8, 20),
       datetime.date(2025, 10, 17), datetime.date(2025, 10, 29),
       datetime.date(2025, 11, 10), datetime.date(2025, 7, 31),
       datetime.date(2025, 7, 30), datetime.date(2025, 8, 5),
       datetime.date(2025, 11, 5), datetime.date(2025, 10, 24),
       datetime.date(2025, 8, 24), dateti

In [114]:

def selecting_data(data):
    display(data)
    links=data['Link']
    times=data['Time']
    accept=[]
    check=lambda t: 'yes' if pd.notna(t) and t >= date.today() - relativedelta(months=1) else 'no'

    for time,link in zip(times,links):
        if check(time)=='yes':
            response=requests.get(link)
            if str(response)=='<Response [200]>':
                accept.append('yes')
            else:
                accept.append('No')
        else:
            accept.append('No')
    data['Accept']=accept
    '''data['Accept'] = data['Time'].apply(
        lambda t: 'yes' if pd.notna(t) and t >= date.today() - relativedelta(months=1) else 'no'
    )'''

    
    df_sorted = data.sort_values(by="Time", ascending=False)
    require = df_sorted[df_sorted['Accept'] == 'yes']

    return require
selected_data=selecting_data(data)
selected_data

,Text,Sub text,Time,Link
0,bharat forge and agile robots sign mou to deploy ai led industrial automation and dark factories,"News: In a press release, Bharat Forge announced it has signed an MoU with Germany-based Agile Robots to jointly advance AI-led industrial...",2026-01-10,https://www.icicidirect.com/research/equity/trending-news/bharat-forge-and-agile-robots-sign-mou-to-deploy-ai-led-industrial-automation-and-dark-factories
1,Bharat Forge partners with Agile to advance AI-led automation,"Driven by Bharat Forge's domain expertise and Agile Robots' intelligent robots and automation solutions, this collaboration will see the...",2026-01-13,https://evertiq.com/design/2026-01-12-bharat-forge-partners-with-agile-to-advance-ai-led-automation
2,Bharat Forge Ltd. is Rated Buy by MarketsMOJO,"Bharat Forge Ltd. is rated Buy by MarketsMOJO, with this rating last updated on 15 Dec 2025. While the rating was revised on that date,...",2026-01-12,https://www.marketsmojo.com/news/stock-recommendation/bharat-forge-ltd-is-rated-buy-by-marketsmojo-3784356
3,Bharat Forge signs strategic MoU with Agile Robots,"Under the MoU, Bharat Forge and Agile Robots will jointly develop customised automation solutions for manufacturing, consumer goods and...",2026-01-11,https://www.manufacturingtodayindia.com/bharat-forge-with-agile-robots
4,"Stocks in news: RIL, BHEL, NTPC, Bharat Forge, Ashok Leyland","Markets closed lower on Thursday due to weak global cues, with the Nifty breaching a key support level. Several companies are in focus...",2026-01-09,https://m.economictimes.com/markets/stocks/news/stocks-in-news-ril-bhel-ntpc-bharat-forge-ashok-leyland/articleshow/126425250.cms
...,...,...,...,...
236,Bharat Forge to Tata Motors: Why did auto stocks fall in trade on Thursday?,"Shares of Bharat Forge and Samvardhana Motherson International fell over 3 per cent each, while Tata Motors and Balkrishna Industries fell over...",2025-07-31,https://www.business-standard.com/markets/news/bharat-forge-to-tata-motors-auto-stocks-fall-up-to-3-per-cent-trump-tariff-threat-125073100274_1.html
237,Bharat Forge acquires AAM India Manufacturing in a Rs 746 crore deal,Bharat Forge has acquired AAM India Manufacturing for Rs 746.4 crore. The acquisition includes the takeover of Rs 189.4 crore cash on the books of AAM India...,2025-07-02,https://www.financialexpress.com/business/industry-bharat-forge-acquires-aam-india-manufacturing-in-a-rs-746-crore-deal-3900474/
238,Nagaland Govt delegation visits Bharat Forge Ltd in Pune,"A delegation from the Nagaland government, including representatives from the Department of Industries & Commerce and the Nagaland Tool Room...",2025-08-23,https://nagalandtribune.in/nagaland-govt-delegation-visits-bharat-forge-ltd-in-pune/
239,Bharat Forge Limited completes Rs 746.46 crore acquisition of AAM India to boost auto and EV components,Bharat Forge Limited (BFL) has successfully completed the acquisition of AAM India Manufacturing Private Limited for an equity value of Rs 746.46 crore.,2025-07-03,https://www.zeebiz.com/companies/news-bharat-forge-limited-completes-rs-74646-crore-acquisition-of-aam-india-to-boost-auto-and-ev-components-371951


,Text,Sub text,Time,Link,Accept
1,Bharat Forge partners with Agile to advance AI-led automation,"Driven by Bharat Forge's domain expertise and Agile Robots' intelligent robots and automation solutions, this collaboration will see the...",2026-01-13,https://evertiq.com/design/2026-01-12-bharat-forge-partners-with-agile-to-advance-ai-led-automation,yes
43,Forged Automotive Components Market to Reach US$ 70.7 Bn by 2032,"Press release - Persistence Market Research - Forged Automotive Components Market to Reach US$ 70.7 Bn by 2032 as Key Players Bharat Forge,...",2026-01-13,https://www.openpr.com/news/4344003/forged-automotive-components-market-to-reach-us-70-7-bn-by-2032,yes
19,Forging Market Booming with Rapid Growth Through 2033| Bharat,"Press release - Coherent Market Insights - Forging Market Booming with Rapid Growth Through 2033| Bharat Forge Limited., Alcoa,...",2026-01-13,https://www.openpr.com/news/4343704/forging-market-booming-with-rapid-growth-through-2033-bharat,yes
2,Bharat Forge Ltd. is Rated Buy by MarketsMOJO,"Bharat Forge Ltd. is rated Buy by MarketsMOJO, with this rating last updated on 15 Dec 2025. While the rating was revised on that date,...",2026-01-12,https://www.marketsmojo.com/news/stock-recommendation/bharat-forge-ltd-is-rated-buy-by-marketsmojo-3784356,yes
15,India's Bharat Forge signs MoU with Germany-based Agile Robots for AI-driven robotics and automation,Indian multinational Bharat Forge and Germany-based Agile Robots SE have signed a memorandum of understanding to explore a strategic...,2026-01-11,https://indianstartupnews.com/news/indias-bharat-forge-signs-mou-with-germany-based-agile-robots-for-ai-driven-robotics-and-automation-10987134,yes
...,...,...,...,...,...
86,"Bharat Forge Limited's (NSE:BHARATFORG) largest shareholders are private companies with 40% ownership, institutions own 40%",Every investor in Bharat Forge Limited (NSE:BHARATFORG) should be aware of the most powerful shareholder groups. And the group that holds the...,2025-12-13,https://simplywall.st/stocks/in/automobiles/nse-bharatforg/bharat-forge-shares/news/bharat-forge-limiteds-nsebharatforg-largest-shareholders-are,yes
159,Bharat Forge - Big relief coming as India-US trade deal inches closer. 10 stocks with upside of 34%,Bharat Forge shows a latest score of 4 with a hold view from 24 analysts. The downside potential is -9%. Institutional holding stands at...,2025-12-13,https://m.economictimes.com/markets/stocks/news/big-relief-coming-as-india-us-trade-deal-inches-closer-10-stocks-with-upside-of-34/bharat-forge/slideshow/125426876.cms,yes
95,Bharat Forge Sees Shift in Market Assessment Amid Mixed Technical and Financial Signals,"Bharat Forge's debt-equity ratio for the half-year period is recorded at a low 0.71 times, reflecting a conservative capital structure and limited reliance on...",2025-12-13,https://www.marketsmojo.com/news/stock-recommendation/bharat-forge-downgraded-to-hold-amid-mixed-technical-and-valuation-signals-3740637,yes
209,Bharat Forge Calls Reuters Story Fake; Denies Link to Russian Delegation Meetings,Bharat Forge has strongly denied a Reuters report claiming its executives secretly met a Russian defence delegation in Moscow. The company...,2025-12-13,https://indianmasterminds.com/news/bharat-forge-reuters-controversy-company-denies-reuters-report-russia-meetings-166959/,yes


In [115]:
def inner_data_(analytic_data):
    driver = webdriver.Chrome()

    link=analytic_data['Link']
    inner_headings = []
    inner_detail = []
    i=0
    j=0
    for i,li in enumerate(link):
        print("=============================================================================================",i)
        print(li)
        try:
            partial_headings = []
            partial_detail = []
    
            driver.get(li)
            time.sleep(5)
    
            #wait = WebDriverWait(driver, 10)
            # -------- H1 TAGS --------
            h1_tags = driver.find_elements(By.TAG_NAME, "h1")
            print(h1_tags)
            for h1 in h1_tags:
                text = h1.text.strip()
                if text:
                    partial_headings.append(text)
                    print('h1:', text)
    
            # -------- P TAGS --------
            p_tags = driver.find_elements(By.TAG_NAME, "p")
            print(p_tags)
            for p in p_tags:
                text = p.text.strip()
                if text:
                    partial_detail.append(text)
                    print('p:', text)
    
            # Always append (even if empty)
            inner_headings.append(" ".join(partial_headings))
            inner_detail.append(partial_detail)
            i+=1
        except:
            j+=1
            print(f"Error in link {li}")
            inner_headings.append("")
            inner_detail.append("")
    print("succ:",i,"fail:",j)
    analytic_data['inner headings'] =inner_headings
    analytic_data['inner detail']=inner_detail
    return analytic_data
data_inner=inner_data_(selected_data[:15])

============================================================================================= 0
https://evertiq.com/design/2026-01-12-bharat-forge-partners-with-agile-to-advance-ai-led-automation
[<selenium.webdriver.remote.webelement.WebElement (session="136627b5477d39b430cd3c37b377a665", element="f.79DBF9831F476602B1494688492C2EA9.d.AEB057116118BA580A7D32F25960F133.e.12")>, <selenium.webdriver.remote.webelement.WebElement (session="136627b5477d39b430cd3c37b377a665", element="f.79DBF9831F476602B1494688492C2EA9.d.AEB057116118BA580A7D32F25960F133.e.13")>, <selenium.webdriver.remote.webelement.WebElement (session="136627b5477d39b430cd3c37b377a665", element="f.79DBF9831F476602B1494688492C2EA9.d.AEB057116118BA580A7D32F25960F133.e.14")>, <selenium.webdriver.remote.webelement.WebElement (session="136627b5477d39b430cd3c37b377a665", element="f.79DBF9831F476602B1494688492C2EA9.d.AEB057116118BA580A7D32F25960F133.e.15")>, <selenium.webdriver.remote.webelement.WebElement (session="136627b5477d39b4

C:\Users\surya\AppData\Local\Temp\ipykernel_12580\3167058295.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analytic_data['inner headings'] =inner_headings
C:\Users\surya\AppData\Local\Temp\ipykernel_12580\3167058295.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analytic_data['inner detail']=inner_detail


In [116]:
 data_inner=data_inner[data_inner['inner headings']!='']
data_inner

Text  \
1                                                   Bharat Forge partners with Agile to advance AI-led automation   
43                                               Forged Automotive Components Market to Reach US$ 70.7 Bn by 2032   
19                                                  Forging Market Booming with Rapid Growth Through 2033| Bharat   
2                                                                   Bharat Forge Ltd. is Rated Buy by MarketsMOJO   
15           India's Bharat Forge signs MoU with Germany-based Agile Robots for AI-driven robotics and automation   
6                              Bharat Forge: Is the Defence Segment Set to Power the Company’s Next Growth Phase?   
31  Bharat Forge Negotiates with European Giants to Relocate Defence Production Lines to India for Global Markets   
17                                      Bharat Forge stock in focus after signing MoU with Germany’s Agile Robots   
0                bharat forge and agile robots sign mou to deploy ai led industrial automation and dark factories   
14                              Bharat Forge, Germany's Agile Robots Sign MoU for AI-Driven Industrial Automation   
27                  Bharat Forge and Agile Robots Enter Strategic Partnership for AI-Driven Industrial Automation   
16                              Bharat Forge, Germany's Agile Robots sign MoU for AI-driven industrial automation   
8                                        Bharat Forge partners Agile Robots for AI-driven robotics and automation   
9                       Bharat Forge inks MoU with Germany’s Agile Robots to explore AI-led industrial automation   
10       Stocks to Watch Today: BHEL, Bajaj Finserv, Bharat Forge, Rail Vikas Nigam, Bharat Electronics, Power...   

                                                                                                                                           Sub text  \
1       Driven by Bharat Forge's domain expertise and Agile Robots' intelligent robots and automation solutions, this collaboration will see the...   
43   Press release - Persistence Market Research - Forged Automotive Components Market to Reach US$ 70.7 Bn by 2032 as Key Players Bharat Forge,...   
19               Press release - Coherent Market Insights - Forging Market Booming with Rapid Growth Through 2033| Bharat Forge Limited., Alcoa,...   
2        Bharat Forge Ltd. is rated Buy by MarketsMOJO, with this rating last updated on 15 Dec 2025. While the rating was revised on that date,...   
15          Indian multinational Bharat Forge and Germany-based Agile Robots SE have signed a memorandum of understanding to explore a strategic...   
6    Bharat Forge is reshaping its growth profile as defence revenues rise from Rs. 410 crore in FY23 to Rs. 1772 crore in FY25, supported by an...   
31        In a significant development for India's defence manufacturing sector, Bharat Forge is currently in advanced negotiations with several...   
17         Bharat Forge has signed an agreement with Germany's Agile Robots. This collaboration will focus on AI-driven robotics and intelligent...   
0     News: In a press release, Bharat Forge announced it has signed an MoU with Germany-based Agile Robots to jointly advance AI-led industrial...   
14               Bharat Forge Limited has signed a strategic MoU with Germany-based Agile Robots S.E. to develop AI-driven robotic solutions for...   
27     Bharat Forge Limited has signed a Memorandum of Understanding (MoU) with Germany-based Agile Robots SE to explore a strategic partnership...   
16     Bharat Forge and Germany-based Agile Robots have signed an MoU to explore a strategic collaboration in AI-driven robotics and intelligent...   
8      Bharat Forge and Germany's Agile Robots have signed an MoU to collaborate on AI-driven robotics and intelligent industrial automation for...   
9   Bharat Forge has signed a MoU with Agile Robots to collaborate on AI-led robotics solutions aimed at civilian manufact

In [117]:
def topic_related_text_extraction(data):
    text=list(data['Text'])
    sub_text=list(data['Sub text'])
    topics=list(data['inner headings'])
    
    texts=list(data['inner detail'])
    extracted_text=[]
    for text_,topic_,mtext_,sub_text_ in zip(texts,topics,text,sub_text):
        print(text_,topic_,"<<============")
        if text_==[] or topic_=='':
            extracted_text.append(mtext_+' '+sub_text_)
            print("in")
        else:
            print("on")
            topic=topic_
            print('topic:',topic)
            ac_li=[]
            end=0
            emb_topic = model.encode(topic, convert_to_tensor=True)

            for i,text in enumerate(text_):
                print(text)
                # Convert to embeddings
                emb_text = model.encode(text, convert_to_tensor=True)
                # Similarity
                similarity = util.cos_sim(emb_text, emb_topic)
                score = similarity[0][0].item()
                if score>0.3:
                    ac_li.append(1)
                   # print(score,'--->',text)
                    end=i
                else:
                    ac_li.append(0)
    
            if 1 in ac_li:
                start=ac_li.index(1)
            else:
                start=end=0
            related=text_[start:end+1]
            #print(related)
            extracted_text.append(mtext_+" "+topic+''.join(related)+" "+sub_text_[:-3])
            print(ac_li)
            
    data['analysis text']=extracted_text
    return data
relavent=topic_related_text_extraction(data_inner)
relavent

['Indian company Bharat Forge and Germany-based Agile Robots have agreed to explore a strategic collaboration to jointly explore the space of AI-driven robotics and intelligent industrial automation.', 'Driven by Bharat Forge’s domain expertise and Agile Robots’ intelligent robots and automation solutions, this collaboration will see the deployment of cutting-edge solutions for the automotive, healthcare, and consumer electronics industries, according to a media release.', 'The Indo-German partnership is focused on civilian industry and manufacturing.', 'To this effect, the companies have signed an MoU to co-develop, set up capabilities and offer bespoke solutions in manufacturing, industrial CPG and logistics space, including existing solutions of Agile Robots for India and Southeast Asia markets; and develop and deploy state-of-the-art vision and AI based robotic solutions as one of the strategic partners to enable a fully autonomous factory, the media release said.', '“This strategi

Text  \
1                                                   Bharat Forge partners with Agile to advance AI-led automation   
43                                               Forged Automotive Components Market to Reach US$ 70.7 Bn by 2032   
19                                                  Forging Market Booming with Rapid Growth Through 2033| Bharat   
2                                                                   Bharat Forge Ltd. is Rated Buy by MarketsMOJO   
15           India's Bharat Forge signs MoU with Germany-based Agile Robots for AI-driven robotics and automation   
6                              Bharat Forge: Is the Defence Segment Set to Power the Company’s Next Growth Phase?   
31  Bharat Forge Negotiates with European Giants to Relocate Defence Production Lines to India for Global Markets   
17                                      Bharat Forge stock in focus after signing MoU with Germany’s Agile Robots   
0                bharat forge and agile robots sign mou to deploy ai led industrial automation and dark factories   
14                              Bharat Forge, Germany's Agile Robots Sign MoU for AI-Driven Industrial Automation   
27                  Bharat Forge and Agile Robots Enter Strategic Partnership for AI-Driven Industrial Automation   
16                              Bharat Forge, Germany's Agile Robots sign MoU for AI-driven industrial automation   
8                                        Bharat Forge partners Agile Robots for AI-driven robotics and automation   
9                       Bharat Forge inks MoU with Germany’s Agile Robots to explore AI-led industrial automation   
10       Stocks to Watch Today: BHEL, Bajaj Finserv, Bharat Forge, Rail Vikas Nigam, Bharat Electronics, Power...   

                                                                                                                                           Sub text  \
1       Driven by Bharat Forge's domain expertise and Agile Robots' intelligent robots and automation solutions, this collaboration will see the...   
43   Press release - Persistence Market Research - Forged Automotive Components Market to Reach US$ 70.7 Bn by 2032 as Key Players Bharat Forge,...   
19               Press release - Coherent Market Insights - Forging Market Booming with Rapid Growth Through 2033| Bharat Forge Limited., Alcoa,...   
2        Bharat Forge Ltd. is rated Buy by MarketsMOJO, with this rating last updated on 15 Dec 2025. While the rating was revised on that date,...   
15          Indian multinational Bharat Forge and Germany-based Agile Robots SE have signed a memorandum of understanding to explore a strategic...   
6    Bharat Forge is reshaping its growth profile as defence revenues rise from Rs. 410 crore in FY23 to Rs. 1772 crore in FY25, supported by an...   
31        In a significant development for India's defence manufacturing sector, Bharat Forge is currently in advanced negotiations with several...   
17         Bharat Forge has signed an agreement with Germany's Agile Robots. This collaboration will focus on AI-driven robotics and intelligent...   
0     News: In a press release, Bharat Forge announced it has signed an MoU with Germany-based Agile Robots to jointly advance AI-led industrial...   
14               Bharat Forge Limited has signed a strategic MoU with Germany-based Agile Robots S.E. to develop AI-driven robotic solutions for...   
27     Bharat Forge Limited has signed a Memorandum of Understanding (MoU) with Germany-based Agile Robots SE to explore a strategic partnership...   
16     Bharat Forge and Germany-based Agile Robots have signed an MoU to explore a strategic collaboration in AI-driven robotics and intelligent...   
8      Bharat Forge and Germany's Agile Robots have signed an MoU to collaborate on AI-driven robotics and intelligent industrial automation for...   
9   Bharat Forge has signed a MoU with Agile Robots to collaborate on AI-led robotics solutions aimed at civilian manufact

pip install ipywidgets

In [118]:
def sentiment_analyse(sentiment_text):




    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(sentiment_text)
    neg = score['neg']
    pos = score['pos']
    neu = score['neu']
    compound = score['compound']

    print(score)
    if compound >= 0.05:
        result='strongly +ve'
    elif compound <= -0.05:
        result='strongly -ve'
    else:
        if neg > pos and neg>=0.4:
            result="-ve"
        elif neg < pos and pos>=0.4:
            result="+ve"
        else:
            result="nue"
    return neg,neu,pos,compound,result

In [31]:
text=relavent['analysis text']
text

9                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [119]:
def final_analysis(data):
    text=data['analysis text']
    print(type(text))
    neg=[]
    neu=[]
    pos=[]
    compound=[]
    result=[]
    for tex in text:
        lower_case=tex.lower()
        cleaned_text = lower_case.translate(str.maketrans('','',string.punctuation))
        s_text=sentiment_analyse(cleaned_text)
        neg_,neu_,pos_,compound_,result_=s_text
        neg.append(neg_)
        neu.append(neu_)
        pos.append(pos_)
        compound.append(compound_)
        result.append(result_)
        print("-----------------------")
    data['Score']=compound
    data['Result']=result
    return data
result_data=final_analysis(relavent)

<class 'pandas.core.series.Series'>
{'neg': 0.009, 'neu': 0.83, 'pos': 0.161, 'compound': 0.9964}
-----------------------
{'neg': 0.0, 'neu': 0.912, 'pos': 0.088, 'compound': 0.3818}
-----------------------
{'neg': 0.075, 'neu': 0.771, 'pos': 0.154, 'compound': 0.6124}
-----------------------
{'neg': 0.02, 'neu': 0.798, 'pos': 0.183, 'compound': 0.9992}
-----------------------
{'neg': 0.0, 'neu': 0.878, 'pos': 0.122, 'compound': 0.9938}
-----------------------
{'neg': 0.017, 'neu': 0.859, 'pos': 0.124, 'compound': 0.9972}
-----------------------
{'neg': 0.0, 'neu': 0.849, 'pos': 0.151, 'compound': 0.6124}
-----------------------
{'neg': 0.037, 'neu': 0.811, 'pos': 0.152, 'compound': 0.9766}
-----------------------
{'neg': 0.0, 'neu': 0.883, 'pos': 0.117, 'compound': 0.9712}
-----------------------
{'neg': 0.034, 'neu': 0.824, 'pos': 0.142, 'compound': 0.9899}
-----------------------
{'neg': 0.014, 'neu': 0.867, 'pos': 0.119, 'compound': 0.9913}
-----------------------
{'neg': 0.0, 'neu

In [120]:
result_data

Text  \
1                                                   Bharat Forge partners with Agile to advance AI-led automation   
43                                               Forged Automotive Components Market to Reach US$ 70.7 Bn by 2032   
19                                                  Forging Market Booming with Rapid Growth Through 2033| Bharat   
2                                                                   Bharat Forge Ltd. is Rated Buy by MarketsMOJO   
15           India's Bharat Forge signs MoU with Germany-based Agile Robots for AI-driven robotics and automation   
6                              Bharat Forge: Is the Defence Segment Set to Power the Company’s Next Growth Phase?   
31  Bharat Forge Negotiates with European Giants to Relocate Defence Production Lines to India for Global Markets   
17                                      Bharat Forge stock in focus after signing MoU with Germany’s Agile Robots   
0                bharat forge and agile robots sign mou to deploy ai led industrial automation and dark factories   
14                              Bharat Forge, Germany's Agile Robots Sign MoU for AI-Driven Industrial Automation   
27                  Bharat Forge and Agile Robots Enter Strategic Partnership for AI-Driven Industrial Automation   
16                              Bharat Forge, Germany's Agile Robots sign MoU for AI-driven industrial automation   
8                                        Bharat Forge partners Agile Robots for AI-driven robotics and automation   
9                       Bharat Forge inks MoU with Germany’s Agile Robots to explore AI-led industrial automation   
10       Stocks to Watch Today: BHEL, Bajaj Finserv, Bharat Forge, Rail Vikas Nigam, Bharat Electronics, Power...   

                                                                                                                                           Sub text  \
1       Driven by Bharat Forge's domain expertise and Agile Robots' intelligent robots and automation solutions, this collaboration will see the...   
43   Press release - Persistence Market Research - Forged Automotive Components Market to Reach US$ 70.7 Bn by 2032 as Key Players Bharat Forge,...   
19               Press release - Coherent Market Insights - Forging Market Booming with Rapid Growth Through 2033| Bharat Forge Limited., Alcoa,...   
2        Bharat Forge Ltd. is rated Buy by MarketsMOJO, with this rating last updated on 15 Dec 2025. While the rating was revised on that date,...   
15          Indian multinational Bharat Forge and Germany-based Agile Robots SE have signed a memorandum of understanding to explore a strategic...   
6    Bharat Forge is reshaping its growth profile as defence revenues rise from Rs. 410 crore in FY23 to Rs. 1772 crore in FY25, supported by an...   
31        In a significant development for India's defence manufacturing sector, Bharat Forge is currently in advanced negotiations with several...   
17         Bharat Forge has signed an agreement with Germany's Agile Robots. This collaboration will focus on AI-driven robotics and intelligent...   
0     News: In a press release, Bharat Forge announced it has signed an MoU with Germany-based Agile Robots to jointly advance AI-led industrial...   
14               Bharat Forge Limited has signed a strategic MoU with Germany-based Agile Robots S.E. to develop AI-driven robotic solutions for...   
27     Bharat Forge Limited has signed a Memorandum of Understanding (MoU) with Germany-based Agile Robots SE to explore a strategic partnership...   
16     Bharat Forge and Germany-based Agile Robots have signed an MoU to explore a strategic collaboration in AI-driven robotics and intelligent...   
8      Bharat Forge and Germany's Agile Robots have signed an MoU to collaborate on AI-driven robotics and intelligent industrial automation for...   
9   Bharat Forge has signed a MoU with Agile Robots to collaborate on AI-led robotics solutions aimed at civilian manufact

In [121]:

result_data['Name']=[name]*len(result_data['Time'])
final_sheet=result_data[['Name','Time','Link','analysis text','Score','Result']]

final_sheet["Link"] = final_sheet["Link"].apply(
    lambda x: f'=HYPERLINK("{x}","Open Link")'
)
'''
df.to_excel(
    "test_result.xlsx",
    index=False,
    engine="xlsxwriter"
)
'''
final_sheet


C:\Users\surya\AppData\Local\Temp\ipykernel_12580\1573010540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_sheet["Link"] = final_sheet["Link"].apply(


Name        Time  \
1   Bharat Forge  2026-01-13   
43  Bharat Forge  2026-01-13   
19  Bharat Forge  2026-01-13   
2   Bharat Forge  2026-01-12   
15  Bharat Forge  2026-01-11   
6   Bharat Forge  2026-01-11   
31  Bharat Forge  2026-01-10   
17  Bharat Forge  2026-01-10   
0   Bharat Forge  2026-01-10   
14  Bharat Forge  2026-01-09   
27  Bharat Forge  2026-01-09   
16  Bharat Forge  2026-01-09   
8   Bharat Forge  2026-01-09   
9   Bharat Forge  2026-01-09   
10  Bharat Forge  2026-01-09   

                                                                                                                                                                                                                                               Link  \
1                                                                                                                     =HYPERLINK("https://evertiq.com/design/2026-01-12-bharat-forge-partners-with-agile-to-advance-ai-led-automation","Open Link")   
43                                                                                                                    =HYPERLINK("https://www.openpr.com/news/4344003/forged-automotive-components-market-to-reach-us-70-7-bn-by-2032","Open Link")   
19                                                                                                                       =HYPERLINK("https://www.openpr.com/news/4343704/forging-market-booming-with-rapid-growth-through-2033-bharat","Open Link")   
2                                                                                                              =HYPERLINK("https://www.marketsmojo.com/news/stock-recommendation/bharat-forge-ltd-is-rated-buy-by-marketsmojo-3784356","Open Link")   
15                                                                        =HYPERLINK("https://indianstartupnews.com/news/indias-bharat-forge-signs-mou-with-germany-based-agile-robots-for-ai-driven-robotics-and-automation-10987134","Open Link")   
6                                                                                                                 =HYPERLINK("https://tradebrains.in/bharat-forge-is-the-defence-segment-set-to-power-the-companys-next-growth-phase/","Open Link")   
31                                                                        =HYPERLINK("https://defence.in/threads/bharat-forge-negotiates-with-european-giants-to-relocate-defence-production-lines-to-india-for-global-markets.16538/","Open Link")   
17                                                                     =HYPERLINK("https://m.economictimes.com/markets/stocks/news/bharat-forge-stock-in-focus-after-signing-mou-with-germanys-agile-robots/articleshow/126428486.cms","Open Link")   
0                                                              =HYPERLINK("https://www.icicidirect.com/research/equity/trending-news/bharat-forge-and-agile-robots-sign-mou-to-deploy-ai-led-industrial-automation-and-dark-factories","Open Link")   
14                                                            =HYPERLINK("https://scanx.trade/stock-market-news/orders-deals/bharat-forge-partners-with-germany-s-agile-robots-for-ai-driven-industrial-automation-solutions/29420781","Open Link")   
27                                                                                        =HYPERLINK("https://themachinemaker.com/news/bharat-forge-and-agile-robots-enter-strategic-partnership-for-ai-driven-industrial-automation/","Open Link")   
16                                                                         =HYPERLINK("https://www.cnbctv18.com/market/stocks/bharat-forge-share-price-germany-agile-robots-sign-mou-for-ai-driven-industrial-automation-19817126.htm","Open Link")   
8                                                                              =HYPERLINK("https://www.thehindubusinessline.com/companies/bharat-forge-agile-robots-sign-mou-for-ai-driven-robotics-collaboration/article70487085.ece","Open Link")   
9                                        

In [122]:

#final_sheet.to_excel('test_result.xlsx',mode='a',header=True)
import os
import pandas as pd

file = "finalize.xlsx"

if os.path.exists(file):
    with pd.ExcelWriter(
        file,
        engine="openpyxl",
        mode="a",
        if_sheet_exists="overlay"
    ) as writer:
        
        startrow = writer.book["Sheet1"].max_row
        
        final_sheet.to_excel(
            writer,
            sheet_name="Sheet1",
            index=False,
            header=False,
            startrow=startrow
        )
else:
    final_sheet.to_excel(file, index=False)


In [40]:
result_data.to_csv('trail.csv',)

In [52]:
vendor_list=pd.read_excel('LT_Vendor_List.xlsx')

In [96]:
vendor_names=vendor_list['Vendor Name']
vendor_names[:10]

0                 Tata Steel
1                  JSW Steel
2                       SAIL
3                Essar Steel
4       Jindal Steel & Power
5           Ultratech Cement
6                    ACC Ltd
7    ISGEC Heavy Engineering
8               Bharat Forge
9      L&T Fabrication Units
Name: Vendor Name, dtype: object

In [45]:
score=[]
result=[]
for name in vendor_names[:2]:
    encoded = quote_plus(name)
    url='http://www.google.com/search?q='+encoded+'&tbm=nws'
    print(url)
    
    score_,result_=sentiment_analysis(name,url)
    score.append(score_)
    result.append(result_)
    print("completed..===============================================================")



http://www.google.com/search?q=Tata+Steel&tbm=nws
http://www.google.com/search?q=Tata+Steel&tbm=nws
raw data:


,Text,Sub text,Time,Link
0,Tata Steel trials Canadian iron ore to hedge against future shortages,Tata Steel has booked its first bulk shipment of iron ore lumps from Tata Steel Minerals Canada to counter potential shortages as captive...,1 day ago,https://www.livemint.com/companies/tata-steel-trials-canadian-iron-ore-to-hedge-against-future-shortages-11767794274252.html
1,Amid uncertainty over IndiaOne Air services JMM leader meets Tata Steel CEO,"Jamshedpur, Jan 8 (PTI) Amid uncertainty over IndiaOne Air-operated flight services from Jamshedpur, senior Jharkhand Mukti Morcha (JMM)...",17 hours ago,https://www.theweek.in/wire-updates/national/2026/01/08/amid-uncertainty-over-indiaone-air-services-jmm-leader-meets-tata-steel-ceo.html
2,"Tata Steel, Jindal Steel, JSW Steel, SAIL stocks: Price targets, outlook ahead of Q3 earnings",ICICI Securities has a buy call on Tata Steel and Jindal Steel. It has a hold call on JSW Steel and SAIL.,24 hours ago,https://www.businesstoday.in/markets/stocks/story/tata-steel-jindal-steel-jsw-steel-sail-stocks-price-targets-outlook-ahead-of-q3-earnings-509999-2026-01-08
3,Varun Bajaj Appointed Chief of Corporate Services of Tata Steel Jamshedpur,Jamshedpur: Tata Steel has announced the appointment of Varun Bajaj as the new Chief Corporate Services (Designate) at its Jamshedpur...,1 day ago,https://avenuemail.in/tata-steel-appoints-varun-bajaj-new-chief-corporate-services-designate-in-jamshedpur/
4,Tata Steel Reports Transfer of Physical Shares and IEPF Transmission,"Tata Steel informs BSE and NSE about physical share transfers and unclaimed shares moved to IEPF as per SEBI regulations, according to its...",2 days ago,https://www.psuconnect.in/corporate-news/tata-steel-reports-transfer-of-physical-shares-and-iepf-transmission


selected data:


,Text,Sub text,Time,Link,Accept
40,"The Tata Steel World 25K Kolkata, with The Telegraph, was a high-octane athletic contest. Snapshots","The Tata Steel World 25K Kolkata, with The Telegraph, was a high-octane athletic contest. Snapshots.",2026-01-09,https://t2online.in/goodlife/sports/the-tata-steel-world-25k-kolkata--with-the-telegraph--was-a-high-octane-athletic-contest--snapshots/2003038,yes
15,TATA STEEL REPORTS STEADY Q3 PRODUCTION GROWTH,3 days ago,2026-01-09,https://www.icicidirect.com/research/equity/trending-news/tata-steel-reports-steady-q3-production-growth,yes
1,Amid uncertainty over IndiaOne Air services JMM leader meets Tata Steel CEO,"Jamshedpur, Jan 8 (PTI) Amid uncertainty over IndiaOne Air-operated flight services from Jamshedpur, senior Jharkhand Mukti Morcha (JMM)...",2026-01-09,https://www.theweek.in/wire-updates/national/2026/01/08/amid-uncertainty-over-indiaone-air-services-jmm-leader-meets-tata-steel-ceo.html,yes
52,Tata Steel shares in focus after company reports best-ever quarterly deliveries in Q3,"Tata Steel reported its best-ever quarterly crude steel production and deliveries for Q3FY26, with production reaching 6.34 million tons,...",2026-01-09,https://m.economictimes.com/markets/stocks/news/tata-steel-shares-in-focus-after-company-reports-best-ever-quarterly-deliveries-in-q3/articleshow/126405221.cms,yes
30,Tata Steel Chess India to become year’s first elite event,Deferment caused by the FIDE World Cup in November 2025 might have come as a blessing in disguise of sorts for Tata Steel Chess India.,2026-01-09,https://revsportz.in/tata-steel-chess-india-to-become-years-first-elite-event/,yes


after selection:


...

clear data:


,Text,Sub text,Time,Link,Accept,Status,inner headings,inner detail,analysis text
0,JSW Steel Q3 output up 6% at 7.48 MT despite Vijayanagar furnace shutdown,JSW Steel on Friday posted a 6 per cent year-on-year rise in consolidated steel output to 7.48 million tonne (MT) in the December quarter.,2026-01-09,https://www.business-standard.com/companies/news/jsw-steel-q3-output-up-6-at-7-48-mt-despite-vijayanagar-furnace-shutdown-126010900449_1.html,yes,working,,,JSW Steel Q3 output up 6% at 7.48 MT despite Vijayanagar furnace shutdownJSW Steel on Friday posted a 6 per cent year-on-year rise in consolidated steel output to 7.48 million tonne (MT) in the December quarter.
11,JSW Steel Consolidated Crude Steel Production Up 6% YoY in Q3 FY26,"JSW Steel reported a 6% YoY increase in consolidated crude steel production for Q3 FY26, reaching 7.48 million tonnes.",2026-01-09,https://www.investywise.com/jsw-steel-reports-consolidated-crude-steel-production-for-q3-fy26/,yes,working,,,"JSW Steel Consolidated Crude Steel Production Up 6% YoY in Q3 FY26JSW Steel reported a 6% YoY increase in consolidated crude steel production for Q3 FY26, reaching 7.48 million tonnes."
40,"Indian steel giants Tata Steel, JSW, others, found in breach of antitrust laws","A confidential CCI order reported by Reuters finds Tata Steel, JSW, and SAIL guilty of price collusion. Top executives like Sajjan Jindal...",2026-01-09,https://www.thebridgechronicle.com/news/india-steel-cartel-cci-finds-tata-jsw-sail-guilty-reuters-as99,yes,working,,,"Indian steel giants Tata Steel, JSW, others, found in breach of antitrust lawsA confidential CCI order reported by Reuters finds Tata Steel, JSW, and SAIL guilty of price collusion. Top executives like Sajjan Jindal..."
21,JSW Steel reports consolidated crude steel production of 7.48MT in Q3 FY26,JSW Steel reported consolidated Crude Steel production for Q3 FY26 at 7.48 million tonnes. The Crude Steel production was higher by 6% YoY.,2026-01-09,https://www.business-standard.com/markets/capital-market-news/jsw-steel-reports-consolidated-crude-steel-production-of-7-48mt-in-q3-fy26-126010900255_1.html,yes,working,,,JSW Steel reports consolidated crude steel production of 7.48MT in Q3 FY26JSW Steel reported consolidated Crude Steel production for Q3 FY26 at 7.48 million tonnes. The Crude Steel production was higher by 6% YoY.
19,JSW Steel registers 6% YoY growth in Q3 FY26 steel production volume,"JSW Steel recorded consolidated steel production of 7.48 million tonnes in Q3 FY26, marking a 6% year-on-year (YoY) increase compared with...",2026-01-09,https://www.business-standard.com/markets/capital-market-news/jsw-steel-registers-6-yoy-growth-in-q3-fy26-steel-production-volume-126010900501_1.html,yes,working,,,"JSW Steel registers 6% YoY growth in Q3 FY26 steel production volumeJSW Steel recorded consolidated steel production of 7.48 million tonnes in Q3 FY26, marking a 6% year-on-year (YoY) increase compared with..."


-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
----------------

,Text,Sub text,Time,Link,Accept,Status,inner headings,inner detail,analysis text,Score,Result
0,JSW Steel Q3 output up 6% at 7.48 MT despite Vijayanagar furnace shutdown,JSW Steel on Friday posted a 6 per cent year-on-year rise in consolidated steel output to 7.48 million tonne (MT) in the December quarter.,2026-01-09,https://www.business-standard.com/companies/news/jsw-steel-q3-output-up-6-at-7-48-mt-despite-vijayanagar-furnace-shutdown-126010900449_1.html,yes,working,,,JSW Steel Q3 output up 6% at 7.48 MT despite Vijayanagar furnace shutdownJSW Steel on Friday posted a 6 per cent year-on-year rise in consolidated steel output to 7.48 million tonne (MT) in the December quarter.,0.0000,nue
11,JSW Steel Consolidated Crude Steel Production Up 6% YoY in Q3 FY26,"JSW Steel reported a 6% YoY increase in consolidated crude steel production for Q3 FY26, reaching 7.48 million tonnes.",2026-01-09,https://www.investywise.com/jsw-steel-reports-consolidated-crude-steel-production-for-q3-fy26/,yes,working,,,"JSW Steel Consolidated Crude Steel Production Up 6% YoY in Q3 FY26JSW Steel reported a 6% YoY increase in consolidated crude steel production for Q3 FY26, reaching 7.48 million tonnes.",-0.6486,strongly -ve
40,"Indian steel giants Tata Steel, JSW, others, found in breach of antitrust laws","A confidential CCI order reported by Reuters finds Tata Steel, JSW, and SAIL guilty of price collusion. Top executives like Sajjan Jindal...",2026-01-09,https://www.thebridgechronicle.com/news/india-steel-cartel-cci-finds-tata-jsw-sail-guilty-reuters-as99,yes,working,,,"Indian steel giants Tata Steel, JSW, others, found in breach of antitrust lawsA confidential CCI order reported by Reuters finds Tata Steel, JSW, and SAIL guilty of price collusion. Top executives like Sajjan Jindal...",0.1280,strongly +ve
21,JSW Steel reports consolidated crude steel production of 7.48MT in Q3 FY26,JSW Steel reported consolidated Crude Steel production for Q3 FY26 at 7.48 million tonnes. The Crude Steel production was higher by 6% YoY.,2026-01-09,https://www.business-standard.com/markets/capital-market-news/jsw-steel-reports-consolidated-crude-steel-production-of-7-48mt-in-q3-fy26-126010900255_1.html,yes,working,,,JSW Steel reports consolidated crude steel production of 7.48MT in Q3 FY26JSW Steel reported consolidated Crude Steel production for Q3 FY26 at 7.48 million tonnes. The Crude Steel production was higher by 6% YoY.,-0.9022,strongly -ve
19,JSW Steel registers 6% YoY growth in Q3 FY26 steel production volume,"JSW Steel recorded consolidated steel production of 7.48 million tonnes in Q3 FY26, marking a 6% year-on-year (YoY) increase compared with...",2026-01-09,https://www.business-standard.com/markets/capital-market-news/jsw-steel-registers-6-yoy-growth-in-q3-fy26-steel-production-volume-126010900501_1.html,yes,working,,,"JSW Steel registers 6% YoY growth in Q3 FY26 steel production volumeJSW Steel recorded consolidated steel production of 7.48 million tonnes in Q3 FY26, marking a 6% year-on-year (YoY) increase compared with...",0.5994,strongly +ve


0.07600289855072465 ------ Positive
completed..===============================================================


In [16]:
df2=pd.DataFrame({'Vendor':vendor_names[:2],'score':score,'result':result})

NameError: name 'vendor_names' is not defined

In [ ]:
df2

In [ ]:
df

In [ ]:
2+2

In [ ]:
pip install transformers

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

text = """
Artificial intelligence enables machines to learn from data,
reason logically, and improve performance over time.
"""

summary = summarizer(text, max_length=30, min_length=10, do_sample=False)
print(summary[0]['summary_text'])
